## Compare pedestals
- Basic plots: pede_diff histogram and 2D map

to use this script, one needs to define the path0/1 and dhes to compare, if necessary, adjust the adc_min/max, vmin/max which defines the axis range of plotting.

In [ ]:
import os
import numpy as np
from matplotlib import pyplot as plt
import mapping
import matplotlib
import epics_utils

dhes = [
                '1012','1082','1022','2042',
                '1062','1072','2052','1042','1052',
                '1071','1061','2051','1041','1051',
                '1011','1031','1081','2041','1021'
                #,'1032'
]
dhes = ['H%s'%dhe for dhe in dhes]

path0 = '/data/commissioning/pedestal_scan/2020_12_16_002/'
path1 = '/data/commissioning/runs/all/EXP0014/pedestal/run2106/'

path=[path0,path1]

for dhe in sorted(dhes):
    path = [os.path.join(path0, dhe), os.path.join(path1, dhe)]
    if not all(os.path.exists(x) for x in path):
        print("[INFO   ] {}: Data not found".format(dhe))
        continue
    peds = [np.load(os.path.join(x, 'analysis.npy'), allow_pickle=True).item() for x in path]
    peds = [np.concatenate([x['PXD:{}:D1:pedestal_data_512'.format(dhe)]
                            ,x['PXD:{}:D2:pedestal_data_512'.format(dhe)]
                            ,x['PXD:{}:D3:pedestal_data_512'.format(dhe)]
                            ,x['PXD:{}:D4:pedestal_data_512'.format(dhe)]
                           ], axis=1)
            for x in peds]
    # plt.figure(figsize=(5, 10))
    module_flavor = epics_utils.get_pv("PXD:B:config-"+dhe,"module_type:VALUE:set").get()
    mapper = mapping.mapper(module_type='pxd9', module_flavor=module_flavor, asicpair=0)
    # ped_diff = mapper(peds[0])
    ped_diff = peds[1].astype(np.int16) - peds[0].astype(np.int16)
    ped_diff_mapped = mapper(ped_diff)
    
    print('mean pede0', np.mean(peds[0]))
    print('mean pede1', np.mean(peds[1]))
    print('diff, mean: %.4f, stfd: %.4f'%(np.mean(ped_diff),np.std(ped_diff)))
    
    fig, axs = plt.subplots(1, 2,figsize=(12, 5))
    fig.suptitle("%s pedestals diff: %s -%s"%(dhe,path1.split('/')[-2],path0.split('/')[-2]), fontsize=16)

    # plotting limits

    adc_min = np.floor(np.amin(ped_diff)) - 0.5
    adc_max = np.ceil(np.amax(ped_diff)) + 0.5
#     adc_min = -20
#     adc_max = 20
    vmax = 3
    vmin = -3
    
    # pedestal distribution as histogram (try stacked plotting)
    try:
        # num_bins = 100
        # prev = np.zeros(num_bins)
        color = ['r', 'y', 'b', 'g']
        color_i = 0

        num_bins = int(np.ceil(adc_max - adc_min))
        prev = np.zeros(num_bins)
        step = max(1, np.floor(float(adc_max - adc_min) / num_bins))
#        print "[DEBUG  ] {}: Pedestal min {:3d}, Pedestal max {:3d}, pedestal step size {}".format(dhe, np.amin(ped_diff), np.amax(ped_di
#            bins = np.arange(np.amin(ped_diff), np.amax(ped_diff)+step*0.1, step)
        bins = np.arange(adc_min, adc_min + step * (num_bins) + 1, step)

        for pedestal_data_i in [(ped_diff[:768, 64 * i:64 * (i + 1)]).flatten() for i in range(4)]:
            n_tmp, bins, patches = axs[0].hist(pedestal_data_i, histtype='bar', bins=bins, fill=True, stacked=False,
                                               alpha=1, width=1,
                                               label="pedestals", align='mid', edgecolor=color[color_i],
                                               bottom=prev,
                                               color=color[color_i])
            prev += n_tmp
            color_i += 1
        axs[0].set_ylim(0.5, np.amax(prev))
        axs[0].set_yscale(r'log')
        axs[0].set_xlabel('ADU')
        axs[0].set_ylabel('Entries')
    except Exception as e:
        print("[WARNING] {}: Could not do stacked pedestals histogram plot due to".format(dhe))
        print(repr(e))

    im = axs[1].imshow(ped_diff_mapped, origin="lower", aspect="auto", vmin=vmin, vmax=vmax)

    axs[1].set_xlabel('Col')
    axs[1].set_ylabel('Row')
    cbar = plt.colorbar(im)
    cbar.ax.set_ylabel('ADU', rotation=270)
    plt.subplots_adjust(wspace=0.3)
    #plt.savefig(os.path.join(args.outputpath, "ped_difference_{}.pdf".format(dhe)), format="pdf")
    plt.show()



## Compare pedestals 
- Plot the pedestals and difference

In [ ]:
import os
import numpy as np
from matplotlib import pyplot as plt
import mapping
import matplotlib
import epics_utils

dhes = [
                '1012','1082','1022','2042',
                '1062','1072','2052','1042','1052',
                '1071','1061','2051','1041','1051',
                '1011','1031','1081','2041','1021'
                #,'1032'
]
dhes = ['H%s'%dhe for dhe in dhes]

path0 = '/data/commissioning/pedestal_scan/2020_12_16_002/'
path1 = '/data/commissioning/runs/all/EXP0014/pedestal/run2106/'

path=[path0,path1]
label0 = path0.split('/')[-2]
label1 = path1.split('/')[-2]

#######################

def plot_pede_hist(ped_diff, ax, adc_min=0, adc_max=255, title=''):
    # pedestal distribution as histogram (try stacked plotting) 
    
    # num_bins = 100
    # prev = np.zeros(num_bins)
    color = ['r', 'y', 'b', 'g']
    color_i = 0

    num_bins = int(np.ceil(adc_max - adc_min))
    prev = np.zeros(num_bins)
    step = max(1, np.floor(float(adc_max - adc_min) / num_bins))

    bins = np.arange(adc_min, adc_min + step * (num_bins) + 1, step)

    for pedestal_data_i in [(ped_diff[:768, 64 * i:64 * (i + 1)]).flatten() for i in range(4)]:
        n_tmp, bins, patches = ax.hist(pedestal_data_i, histtype='bar', bins=bins, fill=True, stacked=False,
                                           alpha=1, width=1,
                                           label="pedestals", align='mid', edgecolor=color[color_i],
                                           bottom=prev,
                                           color=color[color_i])
        prev += n_tmp
        color_i += 1
    ax.set_ylim(0.5, np.amax(prev))
    ax.set_yscale(r'log')
    ax.set_xlabel('ADU')
    ax.set_ylabel('Entries')
    ax.title.set_text(title)
    

def plot_pede_2d(ped_diff, ax, vmin=0, vmax=255, title=''):
        
    module_flavor = epics_utils.get_pv("PXD:B:config-"+dhe,"module_type:VALUE:set").get()
    mapper = mapping.mapper(module_type='pxd9', module_flavor=module_flavor, asicpair=0)
    
    ped_diff_mapped = mapper(ped_diff)
    
    pos = ax.imshow(ped_diff_mapped, origin="lower", aspect="auto", vmin=vmin, vmax=vmax)

    ax.set_xlabel('Col')
    ax.set_ylabel('Row')
    ax.title.set_text(title)

    cbar = fig.colorbar(pos, fraction=0.126,pad=0.02,orientation="vertical")
    cbar.ax.tick_params(labelsize=8, pad=1)
    cbar.ax.set_ylabel('ADU', rotation=270)
    plt.subplots_adjust(wspace=0.3)
    

######################

for dhe in dhes:
    path = [os.path.join(path0, dhe), os.path.join(path1, dhe)]
    if not all(os.path.exists(x) for x in path):
        print("[INFO   ] {}: Data not found".format(dhe))
        continue
    peds = [np.load(os.path.join(x, 'analysis.npy'), allow_pickle=True).item() for x in path]
    peds = [np.concatenate([x['PXD:{}:D1:pedestal_data_512'.format(dhe)]
                            ,x['PXD:{}:D2:pedestal_data_512'.format(dhe)]
                            ,x['PXD:{}:D3:pedestal_data_512'.format(dhe)]
                            ,x['PXD:{}:D4:pedestal_data_512'.format(dhe)]
                           ], axis=1)
            for x in peds]
    
    # plt.figure(figsize=(5, 10))
    module_flavor = epics_utils.get_pv("PXD:B:config-"+dhe,"module_type:VALUE:set").get()
    mapper = mapping.mapper(module_type='pxd9', module_flavor=module_flavor, asicpair=0)
    # ped_diff = mapper(peds[0])
    ped_diff = peds[1].astype(np.int16) - peds[0].astype(np.int16)
    ped_diff_mapped = mapper(ped_diff)
    
    ped_diff_mean = np.mean(ped_diff)
    ped_diff_std = np.std(ped_diff)
    
    print('mean pede0', np.mean(peds[0]))
    print('mean pede1', np.mean(peds[1]))
    print('diff, mean: %.4f, stfd: %.4f'%(ped_diff_mean, ped_diff_std))
    
    fig = plt.figure(figsize=(15, 16))

    fig.suptitle("%s raw pedestals: %s - %s"%(dhe, label1, label0), fontsize=16)


    plot_pede_hist(peds[1], fig.add_subplot(2,3,1), title='%s: %s'%(dhe,label1))
    plot_pede_2d(peds[1], fig.add_subplot(2,3,4),title='%s: %s'%(dhe,label1))
    
    plot_pede_hist(peds[0], fig.add_subplot(2,3,2),title='%s: %s'%(dhe,label0))
    plot_pede_2d(peds[0], fig.add_subplot(2,3,5),title='%s: %s'%(dhe,label0))

    adc_min = np.floor(np.amin(ped_diff)) - 0.5
    adc_max = np.ceil(np.amax(ped_diff)) + 0.5

    vmax = ped_diff_mean+1.5*ped_diff_std
    vmin = ped_diff_mean-1.5*ped_diff_std
#     vmax = 10
#     vmin = -10
    
    plot_pede_hist(ped_diff, fig.add_subplot(2,3,3), adc_min, adc_max, title='%s diff: %s -%s'%(dhe, label1, label0))
    plot_pede_2d(ped_diff, fig.add_subplot(2,3,6), vmin, vmax, title='%s diff: %s -%s'%(dhe, label1, label0))
    
#     if not os.path.exists('ped_diff'):
#         os.makedirs('ped_diff')
#     plt.savefig(os.path.join('ped_diff',"ped_difference_{}.png".format(dhe)))
    
#     plt.show()


